## Hugging Face: Inference to Json (Local)

Common small models that run efficiently on CPUs:
* microsoft/DialoGPT-small (117M, 500MB): basic conversation, very fast on CPU
* gpt2 (124M, 500MB): classic, reliable, fast inference
* distilgpt2 (82M, 350MB): lighter version of GPT-2, faster than GPT-2

Remember to add the environment variable HF_TOKEN containing your token, or login to HF by
> huggingface-cli login

and give the token.

ToDo
* Sample to display information about the model (such as number of parameters)
* Develop script for Full/LORA fine-tuning on json instructions

In [ ]:
import os
import transformers, torch, warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline # For local models
# import torch
from pydantic import BaseModel
# import warnings
# import logging

# Stop the endless flow of nagging
warnings.filterwarnings("ignore")#, message=".*attention mask.*")
# os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = '1'
# logging.getLogger("transformers").setLevel(logging.ERROR)
# logging.set_verbosity_error()

# Global setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device type: " + device.type)
root_folder = r"C:\\temp\\llms"
data_folder = os.path.join(root_folder, "datasets")

Device type: cpu


### Examples for chatting
We show how to use local models for the simplest objective of chatting. We use two versions of the code. The bare version that allows more control, and the version with pipelines for easier syntax.

In [ ]:
# # Single-turn chat through bare interface
# model_name = "meta-llama/Llama-3.2-1B-Instruct"
# # model_name = "distilgpt2"
# # model_name = "microsoft/DialoGPT-small"
# # model_name = "gpt2"
# # model_name = "google/flan-t5-base"
# # model_name = "Qwen/Qwen2.5-1.5B-Instruct"

# # Retrieve local model (download on first call)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# print(f"Model name: {model_name}")
# print(f"Number of parameters: {model.num_parameters():,}")
# print(f"Running on: {model.device}", "\n")

# # Prompt
# prompt = "Who wrote the Bible?"
# input_ids = tokenizer.encode(prompt, return_tensors="pt")

# # Generate response
# outputs = model.generate(input_ids, max_length=25, num_return_sequences=1,
#                          pad_token_id=tokenizer.eos_token_id, temperature=0.7, do_sample=True)

# # Decode
# response = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(response)

Who wrote the Bible? The Bible is a book of sacred scripture that is written by many different authors, including both Old


In [ ]:
# Load the model
model_name = "meta-llama/Llama-3.2-1B-Instruct"
# model_name = "distilgpt2"
# model_name = "microsoft/DialoGPT-small"
# model_name = "gpt2"
# model_name = "google/flan-t5-base"
# model_name = "Qwen/Qwen2.5-1.5B-Instruct"

# Retrieve local model (download on first call)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
print(f"Model name: {model_name}")
print(f"Number of parameters: {model.num_parameters():,}")
print(f"Running on: {model.device}", "\n")

In [ ]:
# Initialize chat history
chat_history_ids = None

# First message
prompt = "Hello! How are you?"
print(f"Prompt: {prompt}")
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate response
chat_history_ids = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id,
                                  temperature=0.7, do_sample=True)

# Decode and print
response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
print(f"Bot: {response}")

# Continue conversation
print()
prompt = "What's your favorite color?"
print(f"Prompt: {prompt}")
input_ids = tokenizer.encode(prompt, return_tensors="pt")
chat_history_ids = torch.cat([chat_history_ids, input_ids], dim=-1)

chat_history_ids = model.generate(chat_history_ids, max_length=200, pad_token_id=tokenizer.eos_token_id)

response = tokenizer.decode(chat_history_ids[:, -50:][0], skip_special_tokens=True)
print(f"Bot: {response}")

Model name: meta-llama/Llama-3.2-1B-Instruct
Number of parameters: 1,235,814,400
Running on: cpu 

Prompt: Hello! How are you?
Bot:  I'm excited to start this new project with you! I've been thinking about how we can improve the user experience on our website.

I'd like to propose a few ideas to enhance the user experience. Firstly, I think we could create a more intuitive navigation menu that makes it easier for users to find what they need. Perhaps we could add a section for frequently asked questions or a "help" section that provides clear instructions on how to use our website.

Additionally

Prompt: What's your favorite color?
Bot:  I was thinking of implementing a new feature that would allow users to save their favorite pages or articles for later. This could be a " favorites" section that allows users to save content for easy access later. Do you think this is something that would be


In [41]:
# Chat through pipeline
chatbot = pipeline("text-generation", model=model_name)

prompt = ["What's your favorite color?", "Tell me a joke"]

# Generate response
response = chatbot(prompt, truncation=True, num_return_sequences=1,
                   pad_token_id=tokenizer.eos_token_id) # max_length=50

for p, r in zip(prompt, response):
    print(f"Prompt: {p}")
    print(f"Bot: {r[0]['generated_text'][len(p):]}")
    print()

Device set to use cpu


Prompt: What's your favorite color?
Bot:  What's your favorite food? What's your favorite hobby? What's your favorite place to visit?
Do you have any pets? What's your favorite type of music?

I'd love to hear about your interests and preferences! It's always great to meet someone who shares similar tastes and passions.

Also, I have to ask: Are you a morning person, a night owl, or somewhere in between?

Prompt: Tell me a joke
Bot: . Why was the math book sad?

(wait for the punchline)

Because it had too many problems!



### Extract json information
Define a json schema for the output. Read a sample email in a text file. Give instruction to the model to extract information from the input email in the json format. Models suitable for instruction have been trained with a specific instruction syntax, which may differ with the model and should be followed for optimal response.

In [43]:
# Define json schema
class OutputSchemaModel(BaseModel):
    customer_name: str
    phone_number: str
    order_number: str
    delivery_address: str

output_schema = OutputSchemaModel.model_json_schema()
print(output_schema)

NameError: name 'BaseModel' is not defined

In [54]:
file = os.path.join(data_folder, "customer_support.txt")
with open(file, "r", encoding='utf-8') as f:
    email = f.read()

print(email)

Subject: Issue with Recent Order #48291

From: emma.johnson@example.com

To: support@shopfast.com

Date: October 26, 2025

Hi ShopFast team,

I placed an order (Order #48291) on October 20, but the package hasn’t arrived yet at 456 Kennedy Ave, 121489 Atlanta, even though the tracking page says “Delivered” since October 23. Could you please check what happened?

Also, I was charged twice for this order on my credit card. Please confirm if I’ll get a refund for the duplicate charge.

Thanks,
Emma Johnson
+44 7911 123456


The first time AutoModelForCausalLM.from_pretrained() is called, it will download the model to the local drive, typically under C:\Users\YourUserName\.cache\huggingface\hub\.

In [10]:
# Json inference
def extract_json(text):
    prompt = f"Extract as JSON: {text}"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [11]:
# Works offline!
result = extract_json("John Doe is 30 years old, email john@example.com")
print(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Extract as JSON: John Doe is 30 years old, email john@example.com, and has a degree in Computer Science. John is married to Jane Doe, who is 28 years old, has a degree in Business, and has a degree in Psychology. John and Jane are both married and have two children, Emily (10) and Michael (7). John has a job at XYZ Corporation, which is a large and well-established company. The company has over 1,000 employees and is headquartered in New York City. John's salary is $120,000 per year. He is also a member of the New York City Police Department, which is responsible for maintaining law and order in the city. John is a member of the local community center and participates in the annual charity event for children's health and education. John has a car and drives a Honda Civic. He enjoys playing basketball and hiking in his free time. John is a big fan of the New York Yankees and attends their games whenever he can. He is a big fan of the New England Patriots and attends their games whenever